# AI Fall 00 - Computer Assignment 1

<div style="font-size: 16px">
<b>Paria Khoshtab 810198387</b>
<hr>
</div>

## Goal

This project aims to get more familiar with `informed` and `uninformed` searching algorithms like `BFS`, `IDS`, `A*`, and `weighted A*`. These search algorithms, give us optimal or suboptimal answers with different execution times.

## Brief Description

In this problem, we have an n*m map whose houses are labeled with `food`, `duplicator` or `wall`.
At first, we have one agent placed in (0, 0), but if a duplicator is eaten by an agent, the agent will be
doubled and the new agent will be placed in (n - 1, 0). Finally, all the foods must be eaten by agents, and 
agents must reach (n - 1, m - 1); we must find a minimum number of moves to reach the mentioned goal.

## Modeling the Problem

The state space of this problem consists of states including information about the position of agents, foods, and duplicators. Also, each state contains information about the parent node and the depth of this node.


**Initial State**: This state consists of initial map of foods and duplicators with an agent placing in (0, 0).
    
    
**Actions**: 
    1) Moving left, right, up and down for one agent at a time 
    2) Eating foods/duplicators

    
**Transition Model**: 1) If you move an agent in any direction, the agent will end up in a new position. 
    2) If an agent eats a food/duplicator, the food/duplicator will be removed from map.

    
**Goal State**:1) There is no more food in the map. 
    2) All the agents are placed in (n - 1, m - 1).

    
**Path Cost**: 1 per move of an agent

First we have to import necessary libraries.

In [1]:
import copy
import collections
import time
import math
import heapq
import bisect

In the code below, we read each line of the input file and save the information about foods,
duplicators and walls in the map.

In [2]:
file = open("Tests/test3.in")
n, m = file.readline().split()
c, k = file.readline().split()
foods = []
duplicators = []
walls = []
for i in range(int(c)):
    x, y = file.readline().split()
    foods.append([int(x), int(y)])

for i in range(int(k)):
    x, y = file.readline().split()
    duplicators.append([int(x), int(y)])
    
d = file.readline()

for i in range(int(d)):
    x, y = file.readline().split()
    walls.append([int(x), int(y)])

We create a class called `Agent` containing the `ID` and `position` of an agent.

In [3]:
class Agent:
    def __init__(self, x, y, ID):
        self.x = x
        self.y = y
        self.ID = ID
    
    def to_string(self):
        return str(self.x) + str(self.y) + str(self.ID)

In each state, we should store the position of agents, foods, and duplicators, the path to reach this state, and the depth of this node in a search tree. So we implement the `State` class as shown below:

In [4]:
class State:
    def __init__(self, ID, agents, foods, duplicators, parent = None, path = None):
        self.ID = ID
        self.agents = agents
        self.foods = foods
        self.duplicators = duplicators
        self.parent = parent
        self.path = path
        if parent == None:
            self.depth = 0
            self.path = ''
        else: 
            self.depth = parent.depth + 1
            
    
    def to_string(self):
        result = ''
        for agent in self.agents:
            result += agent.to_string()
        for food in self.foods:
            result += str(food)
        for dup in self.duplicators:
            result += str(dup)
        return result


We define some necessary functions repeating in all the algorithms:

* **is_goal**: This function checks whether we have reached our goal or not
* **not_wall**: This function checks whether the new position is wall or not
* **get_move**: This function prints the action(moving in any direction)

Also, we define a 2d array for storing directions, called `directions`.

In [5]:
directions = [[1, 0], [-1, 0], [0, 1], [0, -1]]

def is_goal(state):
    for agent in state.agents:
        if agent.x != (int(n) - 1) or agent.y != (int(m) - 1):
            return False
    return len(state.foods) == 0


def not_wall(x, y):
    return x >= 0 and x < int(n) and y >= 0 and y < int(m) and [x, y] not in walls

def get_move(direction, agent):
    result = str(agent.ID)
    if direction == [0, 1]:
        result += 'R'
    elif direction == [1, 0]:
        result += 'U'
    elif direction == [-1, 0]:
        result += 'D'
    elif direction == [0, -1]:
        result += 'L'
    return result

## BFS

The `Breadth First Search` (BFS) is an algorithm for traversing or searching that explores all the nodes at 
the present depth before moving on to the nodes at the next depth level.
This algorithm has a queue as its frontier for keeping track of the child nodes encountered but 
not yet explored and a set as it's explored.

Advantages: BFS is a complete and optimal uninformed search algorithm.
    
* Time complexity: $O(b^d)$
* Space complexity: $O(b^d)$

In [6]:
def BFS(initial_state):
    ID_maker = 1
    explored = set()
    frontier = []
    frontier.append(initial_state)
    if [0, 0] in initial_state.foods:
        initial_state = [i for i in initial_state.foods if i not in [[0, 0]]]
    if [0, 0] in initial_state.duplicators:
        initial_state.duplicators = [i for i in initial_state.duplicators if i not in [[0, 0]]]
        new_agent = Agent(int(n) - 1, 0, 2)
        initial_state.agents.append(new_agent)
    explored.add(initial_state.to_string())
    if(is_goal(initial_state)):
        return initial_state    
    while(len(frontier)):
        state = frontier.pop(0)
        for index in range(len(state.agents)): 
            agent = state.agents[index];
            for d in directions:
                x = agent.x + d[0]
                y = agent.y + d[1]
                if not_wall(x, y):
                    new_agents = copy.deepcopy(state.agents)
                    new_foods = copy.deepcopy(state.foods)
                    new_duplicators = copy.deepcopy(state.duplicators)
                    new_path = copy.deepcopy(state.path) + get_move(d, agent) + ' '
                    new_agents[index].x = x
                    new_agents[index].y = y
                    if [x, y] in state.foods :
                        new_foods = [i for i in new_foods if i not in [[x, y]]]
                    elif [x, y] in state.duplicators:
                        new_duplicators = [i for i in new_duplicators if i not in [[x, y]]]
                        new_agent = Agent(int(n) - 1, 0, len(new_agents) + 1)
                        new_agents.append(new_agent)
                    ID_maker += 1
                    new_state = State(ID_maker, new_agents, new_foods, new_duplicators, state, new_path)
                    if new_state.to_string() not in explored:
                        if is_goal(new_state):
                            return new_state                    
                        frontier.append(new_state)
                        explored.add(new_state.to_string())

Now we initialize the initial state and call the BFS function.

Also, we set an ID for each state, so the ID of the goal state shows us the total number of seen states.

In [7]:
initial_state = State(1, [Agent(0, 0, 1)], foods, duplicators)

In [8]:
start = time.time()
goal_state = BFS(initial_state)
end = time.time()
print("Execution Time: ", end - start)
print("Goal State Depth: ", goal_state.depth)
print("Goal State Path: ", goal_state.path)
print("Number of Seen States: ", goal_state.ID)

Execution Time:  24.29929280281067
Goal State Depth:  19
Goal State Path:  1U 1U 1R 1R 1D 1R 1R 1R 1U 1U 1U 1U 2R 2R 2R 2D 2U 2R 2R 
Number of Seen States:  557667


<table style = "width: 85%; font-size: 14px; margin-left:auto; margin-right:auto;">
    <tr style = "text-align: left">
        <th style = "text-align: left">Test</th>
        <th style = "text-align: left">Goal Depth</th>
        <th style = "text-align: left">Seen States</th>
        <th style = "text-align: left">Mean Execution Time</th>
    </tr>
    <tr>
        <td style = "text-align: left">1</td>
        <td style = "text-align: left">11</td>
        <td style = "text-align: left">3811</td>
        <td style = "text-align: left"> 0.12343207995096843</td>
    </tr>
    <tr>
        <td style = "text-align: left">2</td>
        <td style = "text-align: left">11</td>
        <td style = "text-align: left">494723</td>
        <td style = "text-align: left"> 22.04292933146159</td>
    </tr>
    <tr>
        <td style = "text-align: left">3</td>
        <td style = "text-align: left">19</td>
        <td style = "text-align: left">557667</td>
        <td style = "text-align: left"> 23.191426038742065</td>
    </tr>
</table>

## IDS

`Iterative Deepening Search` (IDS) takes advantage of the completeness of BFS but uses much less memory in
each iteration (like DFS).
IDS calls DLS for different depths starting from an initial value. 
In DLS, we first set a constraint on how deep will we go.<br>
Advantages: IDS is a complete and an optimal uninformed search algorithm that uses much
less memory than BFS.

* Time Complexity: $O(b^d)$
* Space Complexity: $O(bd)$

IDS has a lot more time complexity in this problem than BFS and A*, because goal states have big depth. 

In [9]:
ID_maker = 1
def DLS(state, explored, depth):
    global ID_maker
    explored.add((state.to_string(), state.depth))
    if is_goal(state):
        return state
    elif depth == 0:
        return None  
    for index in range(len(state.agents)): 
        agent = state.agents[index];
        for d in directions:
            x = agent.x + d[0]
            y = agent.y + d[1]
            if not_wall(x, y):
                new_agents = copy.deepcopy(state.agents)
                new_foods = copy.deepcopy(state.foods)
                new_duplicators = copy.deepcopy(state.duplicators)
                new_path = copy.deepcopy(state.path) + get_move(d, agent) + ' '
                new_agents[index].x = x
                new_agents[index].y = y
                if [x, y] in state.foods:
                    new_foods = [i for i in new_foods if i not in [[x, y]]]
                elif [x, y] in state.duplicators:
                    new_duplicators = [i for i in new_duplicators if i not in [[x, y]]]
                    new_agent = Agent(int(n) - 1, 0, len(new_agents) + 1)
                    new_agents.append(new_agent)
                ID_maker += 1
                new_state = State(ID_maker, new_agents, new_foods, new_duplicators, state, new_path)
                if (new_state.to_string(), new_state.depth) not in explored:
                    result = DLS(new_state, explored, depth - 1)
                    if result != None:
                        return result

`DLS` function is an iterative depth-limited searching algorithm with depth as its parameter but IDS function
repeatedly calls DLS function and increases the depth by one unit, until the algorithm reaches the goal.>

In [10]:
def IDS(initial_state):
    depth = 0
    while True:
        explored = set()
        final_state = DLS(initial_state, explored, depth)
        if final_state != None:
            break
        depth += 1       
    return final_state

In [11]:
start = time.time()
goal_state = IDS(initial_state)
end = time.time()
print("Execution Time: ", end - start)
print("Goal State Depth: ", goal_state.depth)
print("Goal State Path: ", goal_state.path)
print("Number of Seen States: ", goal_state.ID)

Execution Time:  95.30243825912476
Goal State Depth:  19
Goal State Path:  1U 1U 1R 1R 1D 1R 1R 1R 1U 1U 1U 1U 2R 2R 2R 2D 2U 2R 2R 
Number of Seen States:  2401959


<table style = "width: 85%; font-size: 14px; margin-left:auto; margin-right:auto;">
    <tr style = "text-align: left">
        <th style = "text-align: left">Test</th>
        <th style = "text-align: left">Goal Depth</th>
        <th style = "text-align: left">Seen States</th>
        <th style = "text-align: left">Mean Execution Time</th>
    </tr>
    <tr>
        <td style = "text-align: left">1</td>
        <td style = "text-align: left">11</td>
        <td style = "text-align: left">23442</td>
        <td style = "text-align: left"> 0.6954633394877116</td>
    </tr>
    <tr>
        <td style = "text-align: left">2</td>
        <td style = "text-align: left">11</td>
        <td style = "text-align: left">1260901</td>
        <td style = "text-align: left"> 52.690138816833496</td>
    </tr>
    <tr>
        <td style = "text-align: left">3</td>
        <td style = "text-align: left">19</td>
        <td style = "text-align: left">2401959</td>
        <td style = "text-align: left"> 92.9502682685852</td>
    </tr>
</table>

## A*

`A*` is an extension of `Dijkstra's algorithm` with some characteristics of `BFS`; This avoids expanding paths that are already expensive.
The A* algorithm introduces a heuristic, essentially planning ahead at each step so a more optimal decision is made.
The evaluation function f(n) is the estimated total cost of the path through node n to the goal:

f(n) = g(n) + h(n)

g(n): cost so far to reach n(path cost)

h(n): estimated cost from n to goal(heuristic)

Advantages: A* is a complete and optimal(if h(n) is admissible/consistent) uninformed search algorithm that 
is usually faster than IDS and BFS.

* Time Complexity: number of nodes which heuristic expands
* Space Complexity: exponential

The heuristic function we use here is the maximum Manhattan distance from an agent position to the destination(n - 1, m - 1)

The depth of our current state is our cost so far to reach n.
Our heuristic function is **Consistent**.

**Prove**: From the definition, we know a heuristic function is consistent if:
We have state B after state A and the real cost of A to B is greater than or equal to the cost implied by heuristics.
So it is obvious that in our heuristic, h(B) is at most one unit more than h(A) because each agent is only allowed
one unit per move in any direction, but the cost(A to B) is at least one because the minimum depth between two states
is at least one, so we have: cost(A to B) >= h(B)-h(A)

In [12]:
def get_manhattan_heuristic(state):
    max_distance = -1
    for agent in state.agents:
        if max_distance < (abs(int(m) - agent.y)) + (abs(int(n) - agent.x)):
            max_distance = (abs(int(m) - agent.y)) + (abs(int(n) - agent.x))
    return max_distance

We create priority queue class as shown below to implement the approriate data structure as froniter in A* algorithm.
In this data structure you can pop the node with highest priority in O(1).

In [13]:
class PriorityQueue:
    def __init__(self):
        self.queue = []
        self.index = 0
 
    def push(self, item, priority):
        bisect.insort(self.queue, (priority, self.index, item))
        self.index += 1
 
    def pop(self):
        return heapq.heappop(self.queue)[-1]
    
    def is_empty(self):
        return len(self.queue) == 0

In [14]:
def AStar(initial_state):
    ID_maker = 1
    explored = set()
    frontier = PriorityQueue()
    frontier.push(initial_state, get_manhattan_heuristic(initial_state) + initial_state.depth)
    if [0, 0] in initial_state.foods:
        initial_state = [i for i in initial_state.foods if i not in [[0, 0]]]
    if [0, 0] in initial_state.duplicators:
        initial_state.duplicators = [i for i in initial_state.duplicators if i not in [[0, 0]]]
        new_agent = Agent(int(n) - 1, 0, 2)
        initial_state.agents.append(new_agent)
    explored.add(initial_state.to_string())
    if(is_goal(initial_state)):
        return initial_state    
    while not frontier.is_empty():
        state = frontier.pop()
        for index in range(len(state.agents)): 
            agent = state.agents[index];
            for d in directions:
                x = agent.x + d[0]
                y = agent.y + d[1]
                if not_wall(x, y):
                    new_agents = copy.deepcopy(state.agents)
                    new_foods = copy.deepcopy(state.foods)
                    new_duplicators = copy.deepcopy(state.duplicators)
                    new_path = copy.deepcopy(state.path) + get_move(d, agent) + ' '
                    new_agents[index].x = x
                    new_agents[index].y = y
                    if [x, y] in state.foods:
                        new_foods = [i for i in new_foods if i not in [[x, y]]]
                    elif [x, y] in state.duplicators:
                        new_duplicators = [i for i in new_duplicators if i not in [[x, y]]]
                        new_agent = Agent(int(n) - 1, 0, len(new_agents) + 1)
                        new_agents.append(new_agent)
                    ID_maker += 1
                    new_state = State(ID_maker, new_agents, new_foods, new_duplicators, state, new_path)
                    if new_state.to_string() not in explored:
                        if is_goal(new_state):
                            return new_state                    
                        frontier.push(new_state, get_manhattan_heuristic(new_state) + new_state.depth)
                        explored.add(new_state.to_string())    

In [15]:
start = time.time()
goal_state = AStar(initial_state)
end = time.time()
print("Execution Time: ", end - start)
print("Goal State Depth: ", goal_state.depth)
print("Goal State Path: ", goal_state.path)
print("Number of Seen States: ", goal_state.ID)

Execution Time:  3.012855052947998
Goal State Depth:  19
Goal State Path:  1U 1U 1R 1R 2R 1D 1R 2R 2R 1R 1R 1U 2D 2R 1U 1U 2U 2R 1U 
Number of Seen States:  71143


<table style = "width: 85%; font-size: 14px; margin-left:auto; margin-right:auto;">
    <tr style = "text-align: left">
        <th style = "text-align: left">Test</th>
        <th style = "text-align: left">Goal Depth</th>
        <th style = "text-align: left">Seen States</th>
        <th style = "text-align: left">Mean Execution Time</th>
    </tr>
    <tr>
        <td style = "text-align: left">1</td>
        <td style = "text-align: left">11</td>
        <td style = "text-align: left">2030</td>
        <td style = "text-align: left"> 0.09650699297587077</td>
    </tr>
    <tr>
        <td style = "text-align: left">2</td>
        <td style = "text-align: left">11</td>
        <td style = "text-align: left">38120</td>
        <td style = "text-align: left"> 1.9440749486287434</td>
    </tr>
    <tr>
        <td style = "text-align: left">3</td>
        <td style = "text-align: left">19</td>
        <td style = "text-align: left">71143</td>
        <td style = "text-align: left"> 3.2238033612569175</td>
    </tr>
</table>

## Weighted A*

`Weighted A*` is a variant of A* commonly used for suboptimal search.
It is exactly the same as A* butwhere the f-value is computed differently: f(n) = g(n) + alpha*h(n)


Weighted A* search algorithm speedup search at expense of optimality.
This algorithm multiplies the admissible heuristic by some constant factor(alpha > 1), and then performs an A* search as usual.


Although weighted A* expands fewer nodes, there is no guarantee that it will necessarily
find the optimal solution.

In [16]:
def weighted_AStar(initial_state):
    alpha = 1.5
#     alpha = 1.8
    ID_maker = 1
    explored = set()
    frontier = PriorityQueue()
    frontier.push(initial_state, get_manhattan_heuristic(initial_state)*alpha + initial_state.depth)
    if [0, 0] in initial_state.foods:
        initial_state = [i for i in initial_state.foods if i not in [[0, 0]]]
    if [0, 0] in initial_state.duplicators:
        initial_state.duplicators = [i for i in initial_state.duplicators if i not in [[0, 0]]]
        new_agent = Agent(int(n) - 1, 0, 2)
        initial_state.agents.append(new_agent)
    explored.add(initial_state.to_string())
    if(is_goal(initial_state)):
        return initial_state    
    while not frontier.is_empty():
        state = frontier.pop()
        for index in range(len(state.agents)): 
            agent = state.agents[index];
            for d in directions:
                x = agent.x + d[0]
                y = agent.y + d[1]
                if not_wall(x, y):
                    new_agents = copy.deepcopy(state.agents)
                    new_foods = copy.deepcopy(state.foods)
                    new_duplicators = copy.deepcopy(state.duplicators)
                    new_path = copy.deepcopy(state.path) + get_move(d, agent) + ' '
                    new_agents[index].x = x
                    new_agents[index].y = y
                    if [x, y] in state.foods:
                        new_foods = [i for i in new_foods if i not in [[x, y]]]
                    elif [x, y] in state.duplicators:
                        new_duplicators = [i for i in new_duplicators if i not in [[x, y]]]
                        new_agent = Agent(int(n) - 1, 0, len(new_agents) + 1)
                        new_agents.append(new_agent)
                    ID_maker += 1
                    new_state = State(ID_maker, new_agents, new_foods, new_duplicators, state, new_path)
                    if new_state.to_string() not in explored:
                        if is_goal(new_state):
                            return new_state                    
                        frontier.push(new_state, get_manhattan_heuristic(new_state)*alpha + new_state.depth)
                        explored.add(new_state.to_string())    

In [17]:
start = time.time()
goal_state = weighted_AStar(initial_state)
end = time.time()
print("Execution Time: ", end - start)
print("Goal State Depth: ", goal_state.depth)
print("Goal State Path: ", goal_state.path)
print("Number of Seen States: ", goal_state.ID)

Execution Time:  0.7893800735473633
Goal State Depth:  19
Goal State Path:  1U 1U 1R 1R 1D 1R 1R 1R 2R 2R 1U 2R 1U 1U 2D 2R 2R 1U 2U 
Number of Seen States:  18274


<table style = "width: 85%; font-size: 14px; margin-left:auto; margin-right:auto;">
    <tr style = "text-align: left">
        <th style = "text-align: left">Test</th>
        <th style = "text-align: left">Alpha</th>
        <th style = "text-align: left">Goal Depth</th>
        <th style = "text-align: left">Seen States</th>
        <th style = "text-align: left">Mean Execution Time</th>
    </tr>
    <tr>
        <td style = "text-align: left">1</td>
        <td style = "text-align: left">1.5</td>
        <td style = "text-align: left">11</td>
        <td style = "text-align: left">1262</td>
        <td style = "text-align: left"> 0.05303311347961426</td>
    </tr>
    </tr>
        <tr>
        <td style = "text-align: left">1</td>
        <td style = "text-align: left">1.8</td>
        <td style = "text-align: left">11</td>
        <td style = "text-align: left">743</td>
        <td style = "text-align: left"> 0.035580952962239586</td>
    </tr>
    <tr>
        <td style = "text-align: left">2</td>
        <td style = "text-align: left">1.5</td>
        <td style = "text-align: left">11</td>
        <td style = "text-align: left">7006</td>
        <td style = "text-align: left"> 0.3737959861755371</td>
    </tr>
    <tr>
        <td style = "text-align: left">2</td>
        <td style = "text-align: left">1.8</td>
        <td style = "text-align: left">11</td>
        <td style = "text-align: left">2838</td>
        <td style = "text-align: left"> 0.17054128646850586</td>
    </tr>
    <tr>
        <td style = "text-align: left">3</td>
        <td style = "text-align: left">1.5</td>
        <td style = "text-align: left">19</td>
        <td style = "text-align: left">18274</td>
        <td style = "text-align: left"> 0.8635300000508627</td>
    <tr>
        <td style = "text-align: left">3</td>
        <td style = "text-align: left">1.8</td>
        <td style = "text-align: left">19</td>
        <td style = "text-align: left">12325</td>
        <td style = "text-align: left"> 0.5862730344136556</td>
    </tr>
</table>

## Conclusion

here are a lot of searching algorithms, grouped into informed and uninformed that we can use the desired one
depending on our needs (time complexity, space complexity, optimality, ...).

In general, 
between both informed and uninformed search techniques, the informed search is more efficient and cost-effective in
terms of space and time complexity, because informed search algorithms use "domain knowledge" (heuristic function). 

## References

https://www.educative.io/edpresso/what-is-iterative-deepening-search<br>
https://python.plainenglish.io/how-to-easily-implement-priority-queue-in-python-ceabba729d96<br>    
